# Data preparation/downloading/processing notebook

### First we need to move to the top-level directory.

In [ ]:
cd ../..

## Downloading preprocessed data

This notebook works with a sub-sample of the natural question collection (__Wikipedia DPR__) prepared by [Karpukhin et al.](https://github.com/facebookresearch/DPR). This subset includes all the questions, but only about one million Wikipedia passages. The generation of this subset is briefly described below, but for your convenience we provide an archive with already processed data:

In [2]:
!wget boytsov.info/datasets/wikipedia_dpr_nq_sample_2021-06-30.tar.bz2

--2021-07-01 14:37:38--  http://boytsov.info/datasets/wikipedia_dpr_nq_sample_2021-06-30.tar.bz2
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2766256898 (2.6G) [application/x-bzip2]
Saving to: ‘wikipedia_dpr_nq_sample_2021-06-30.tar.bz2’

wikipedia_dpr_nq_sa 100%[===================>]   2.58G  16.8MB/s    in 3m 15s  

2021-07-01 14:40:53 (13.5 MB/s) - ‘wikipedia_dpr_nq_sample_2021-06-30.tar.bz2’ saved [2766256898/2766256898]



In [6]:
!tar jxvf wikipedia_dpr_nq_sample_2021-06-30.tar.bz2

x wikipedia_dpr_nq_sample/input_data/
x wikipedia_dpr_nq_sample/input_data/train_fusion/
x wikipedia_dpr_nq_sample/input_data/train_fusion/QuestionFields.jsonl
x wikipedia_dpr_nq_sample/input_data/train_fusion/qrels.txt
x wikipedia_dpr_nq_sample/input_data/train_fusion/QuestionFields.bin
x wikipedia_dpr_nq_sample/input_data/dev/
x wikipedia_dpr_nq_sample/input_data/dev/QuestionFields.jsonl
x wikipedia_dpr_nq_sample/input_data/dev/qrels.txt
x wikipedia_dpr_nq_sample/input_data/dev/QuestionFields.bin
x wikipedia_dpr_nq_sample/input_data/bitext/
x wikipedia_dpr_nq_sample/input_data/bitext/QuestionFields.jsonl
x wikipedia_dpr_nq_sample/input_data/bitext/qrels.txt
x wikipedia_dpr_nq_sample/input_data/pass_sample/
x wikipedia_dpr_nq_sample/input_data/pass_sample/AnswerFields.bin
x wikipedia_dpr_nq_sample/input_data/pass_sample/AnswerFields.jsonl.gz
x wikipedia_dpr_nq_sample/input_data/dev_official/
x wikipedia_dpr_nq_sample/input_data/dev_official/QuestionFields.jsonl
x wikipedia_dpr_nq_samp

In [ ]:
!mkdir collections

In [7]:
!mv wikipedia_dpr_nq_sample collections

#### Carry out a basic sanity check:

In [8]:
!scripts/report/get_basic_collect_stat.sh wikipedia_dpr_nq_sample

Using collection root: collections
Checking input sub-directory: bitext
Checking input sub-directory: dev
Checking input sub-directory: dev_official
Checking input sub-directory: pass_sample
Found indexable data file: pass_sample/AnswerFields.jsonl.gz
Checking input sub-directory: train_fusion
Found query file: bitext/QuestionFields.jsonl
Found query file: dev/QuestionFields.jsonl
Found query file: dev_official/QuestionFields.jsonl
Found query file: train_fusion/QuestionFields.jsonl
getIndexQueryDataDirs return value:  pass_sample AnswerFields.jsonl.gz bitext,dev,dev_official,train_fusion
Using data file: AnswerFields.jsonl.gz
Index dirs: pass_sample
Query dirs: bitext dev dev_official train_fusion
Queries/questions:
bitext 53880
dev 2500
dev_official 6515
train_fusion 2500
Documents/passages/answers:
pass_sample 774392


## Preprocessing in more details
### Most users can ignore this part (at least the first time the study these notebooks) and proceed to notebooks that describe indexing and model training. 

The download and conversion script can be found in the directory `scripts/data_convert/wikipedia_dpr`.

In [ ]:
!mkdir -p collections/wikipedia_dpr_nq_sample/input_raw

### Download passages and queries

In [ ]:
!scripts/data_convert/wikipedia_dpr/download_dpr_passages.sh collections/wikipedia_dpr_nq_sample/input_raw

In [ ]:
!scripts/data_convert/wikipedia_dpr/download_dpr_queries.sh nq collections/wikipedia_dpr_nq_sample/input_raw

### Randomly split the training set into the new training and development sets. This script also converts the data into FlexNeuART format

In [ ]:
!scripts/data_convert/wikipedia_dpr/split_and_convert_dpr_queries.sh \
    wikipedia_dpr_nq_sample \
    collections/wikipedia_dpr_nq_sample/input_raw/ \
    nq \
    -partition_sizes ,5000,2500 

### The split & convert script produces outputs of two types:
1. The set of questions in JSONL format. These questions are divided into several subsets:

In [9]:
!ls collections/wikipedia_dpr_nq_sample/input_data

bitext       dev          dev_official pass_sample  train_fusion


The `bitext` subset and the `train_fusion` subsets are supposed to be used to train models. The difference is that `train_fusion` is a smaller subset that can be used to create fusion models. The `bitext` part can be used to train, e.g., neural models.

For the queries from the `bitext` set, the conversion script creates parallel data (bitext) where questions are aligned with respective answer-bearing sentences. We create three parallel corpora that correspond to three ways to lemmatize & tokenize input (lemmas and original tokens with stopwords removed and BERT-tokenized text). They are stored in the `derived_data/bitext` subdirectory:

In [10]:
!ls collections/wikipedia_dpr_nq_sample/derived_data/bitext

answer_text_bert_tok   question_text_bert_tok
answer_text_unlemm     question_text_unlemm


### Embedding document and queries

1. We already __ship__ data with documents and queries (except for the bitext part) embedded using an [ANCE Wikipedia model](https://github.com/microsoft/ANCE). This is done using the scripts in the `scripts/data_convert/ance` directory.
2. First, one needs to download the models using the script `scripts/data_convert/ance/download_ance_models.sh`.
3. Then, one can embed documents using a command like this one:

```
scripts/data_convert/ance/embed.py \
    --input collections/wikipedia_dpr_nq_sample/input_raw/psgs_w100.tsv.gz \
    --output collections/wikipedia_dpr_nq_sample/input_data/pass_sample/AnswerFields.bin \
    --field_name dense  \
    --model_dir <model download directory> \
    --data_type dpr_nq \
    --doc_ids collections/wikipedia_dpr_nq_sample/input_raw/nq_selected_psg_ids.npy
```

4. ... and queries using a command like this one (note we specify __the binary field name__):

```
for part in train_fusion dev dev_official ; do \
    scripts/data_convert/ance/embed.py \
        --input collections/wikipedia_dpr_nq_sample/input_data/$part/QuestionFields.jsonl \
        --output collections/wikipedia_dpr_nq_sample/input_data/$part/QuestionFields.bin \
        --field_name dense  \
        --model_dir <model download directory> \
        --data_type dpr_nq 
done
```